In [14]:
import math
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.mllib.recommendation import Rating
from pyspark.mllib.recommendation import ALS

SparkContext.setSystemProperty('spark.executor.memory', '3g')
spark_session = SparkSession.builder \
    .enableHiveSupport() \
    .appName("recSys")\
    .master("local[8]")\
    .getOrCreate()
    
sc = spark_session.sparkContext
sc.setCheckpointDir('/tmp')

data = spark_session.read.format("csv").option("header", "true").load("/data/movielens/ratings_100k.csv")

split = data.rdd.map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2]))).randomSplit([0.333,0.333,0.334])

In [15]:
l = []
rank = 20
iterations = 5
lambda_ = 0.01

test = split[0].map(lambda t: (t[0], t[1]))
train = split[1].union(split[2])

model = ALS.train(train,rank =rank, iterations = iterations, lambda_ = lambda_,blocks = 15)
predictions = model.predictAll(test).map(lambda r: ((r[0], r[1]), r[2]))

ratesAndPreds = split[0].map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
mse = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()

l.append(math.sqrt(mse))

In [16]:
score = sum(l) / len(l)

In [17]:
print(score)

1.30774496651
